In [1]:
import pandas as pd
import sqlite3 as sql
import numpy as np
from tqdm import tqdm

# 1. Crime Amount

In [2]:
# Create connection to the crime SQL database that is made before
conn = sql.connect('crime.db')

In [3]:
# Select number of crime per month per crime type per LSOA
query_crime_type_LSOA = '''
    select Year, Month, Section, LSOA_code, LSOA_name, Crime_type, count(Longitude) 'Amount'
    from street_crime
    group by Year, Month, Section, LSOA_code, LSOA_name, Crime_type
'''
# This query runs for 5-6 minutes
df_LSOA = pd.read_sql_query(query_crime_type_LSOA, conn)
# Make sure LSOA name exists for all columns so that we can perform analysis
df_LSOA_clean = df_LSOA[~df_LSOA['LSOA_name'].isna()]
# Remove section btp and metropolitan because they rule over the entire UK, not a specific province with LSOA code
df_LSOA_need = df_LSOA_clean[(df_LSOA_clean['Section'] != 'btp') & (df_LSOA_clean['Section'] != 'metropolitan')]
df_LSOA_need

,Year,Month,Section,LSOA_code,LSOA_name,Crime_type,Amount
0,2010,12,avon-and-somerset,E01012168,Redcar and Cleveland 022C,Vehicle crime,1
1,2010,12,avon-and-somerset,E01014370,Bath and North East Somerset 007A,Anti-social behaviour,63
2,2010,12,avon-and-somerset,E01014370,Bath and North East Somerset 007A,Burglary,2
3,2010,12,avon-and-somerset,E01014370,Bath and North East Somerset 007A,Other crime,65
4,2010,12,avon-and-somerset,E01014370,Bath and North East Somerset 007A,Robbery,1
...,...,...,...,...,...,...,...
21935634,2021,10,wiltshire,E01032716,Swindon 012G,Other crime,2
21935635,2021,10,wiltshire,E01032716,Swindon 012G,Public order,2
21935636,2021,10,wiltshire,E01032716,Swindon 012G,Shoplifting,8
21935637,2021,10,wiltshire,E01032716,Swindon 012G,Vehicle crime,1


# 2. Calculate overall total crime per LSOA

In [4]:
# Define the main crime types I and II
type_I = ['Vehicle crime', 'Violent crime', 'Robbery', 'Criminal damage and arson', 'Violence and sexual offences']
type_II = ['Burglary', 'Other crime', 'Drugs', 'Other theft', 'Public disorder and weapons', 'Shoplifting', 
           'Bicycle theft', 'Public order', 'Theft from the person', 'Possession of weapons']

In [5]:
# Classify each row with specific crime type to crime type I and II
df_LSOA_need['Main_crime'] = np.where(df_LSOA_need['Crime_type'].isin(type_I), 'Type I', 'Type II')
df_LSOA_need

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Year,Month,Section,LSOA_code,LSOA_name,Crime_type,Amount,Main_crime
0,2010,12,avon-and-somerset,E01012168,Redcar and Cleveland 022C,Vehicle crime,1,Type I
1,2010,12,avon-and-somerset,E01014370,Bath and North East Somerset 007A,Anti-social behaviour,63,Type II
2,2010,12,avon-and-somerset,E01014370,Bath and North East Somerset 007A,Burglary,2,Type II
3,2010,12,avon-and-somerset,E01014370,Bath and North East Somerset 007A,Other crime,65,Type II
4,2010,12,avon-and-somerset,E01014370,Bath and North East Somerset 007A,Robbery,1,Type I
...,...,...,...,...,...,...,...,...
21935634,2021,10,wiltshire,E01032716,Swindon 012G,Other crime,2,Type II
21935635,2021,10,wiltshire,E01032716,Swindon 012G,Public order,2,Type II
21935636,2021,10,wiltshire,E01032716,Swindon 012G,Shoplifting,8,Type II
21935637,2021,10,wiltshire,E01032716,Swindon 012G,Vehicle crime,1,Type I


In [11]:
# Find the total number of type I and type II crime per month per year
df_need = df_LSOA_need.groupby(['Year', 'Month', 'LSOA_code', 'LSOA_name', 'Main_crime']).sum()[['Amount']].reset_index()
df_need.rename(columns = {'Amount': 'Crime_amount'}, inplace = True)
df_need

,Year,Month,LSOA_code,LSOA_name,Main_crime,Crime_amount
0,2010,12,E01000001,City of London 001A,Type I,1
1,2010,12,E01000001,City of London 001A,Type II,3
2,2010,12,E01000002,City of London 001B,Type I,7
3,2010,12,E01000002,City of London 001B,Type II,20
4,2010,12,E01000003,City of London 001C,Type I,1
...,...,...,...,...,...,...
7129673,2021,10,W01001956,Swansea 023E,Type II,2
7129674,2021,10,W01001957,Swansea 025G,Type I,13
7129675,2021,10,W01001957,Swansea 025G,Type II,24
7129676,2021,10,W01001958,Swansea 025H,Type I,7


In [12]:
# Select data after 2011
df_need = df_need[df_need['Year'] >= 2011]
df_need

,Year,Month,LSOA_code,LSOA_name,Main_crime,Crime_amount
50830,2011,1,E01000001,City of London 001A,Type I,2
50831,2011,1,E01000001,City of London 001A,Type II,8
50832,2011,1,E01000002,City of London 001B,Type I,2
50833,2011,1,E01000002,City of London 001B,Type II,17
50834,2011,1,E01000003,City of London 001C,Type I,3
...,...,...,...,...,...,...
7129673,2021,10,W01001956,Swansea 023E,Type II,2
7129674,2021,10,W01001957,Swansea 025G,Type I,13
7129675,2021,10,W01001957,Swansea 025G,Type II,24
7129676,2021,10,W01001958,Swansea 025H,Type I,7


In [13]:
# Find the cumulated month of each record, with 2011-1 as the beginning month
df_need['Total_month'] = (df_need['Year'] - df_need['Year'].min()) * 12 + df_need['Month']
df_need

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Year,Month,LSOA_code,LSOA_name,Main_crime,Crime_amount,Total_month
50830,2011,1,E01000001,City of London 001A,Type I,2,1
50831,2011,1,E01000001,City of London 001A,Type II,8,1
50832,2011,1,E01000002,City of London 001B,Type I,2,1
50833,2011,1,E01000002,City of London 001B,Type II,17,1
50834,2011,1,E01000003,City of London 001C,Type I,3,1
...,...,...,...,...,...,...,...
7129673,2021,10,W01001956,Swansea 023E,Type II,2,130
7129674,2021,10,W01001957,Swansea 025G,Type I,13,130
7129675,2021,10,W01001957,Swansea 025G,Type II,24,130
7129676,2021,10,W01001958,Swansea 025H,Type I,7,130


In [14]:
# Create 2 columns to show type I and type II crime amounts at once
# This allows easier comparison and aggregation of data
df_I = df_need[df_need['Main_crime'] == 'Type I'].reset_index(drop = True).drop(columns = 'Main_crime').rename(columns = {'Crime_amount': 'Type_I_crime_amount'})
df_II = df_need[df_need['Main_crime'] == 'Type II'].reset_index(drop = True).drop(columns = 'Main_crime').rename(columns = {'Crime_amount': 'Type_II_crime_amount'})
df_amount = pd.merge(df_I, df_II, on = ['Year', 'Month', 'Total_month', 'LSOA_code', 'LSOA_name'], how = 'inner')
df_amount

,Year,Month,LSOA_code,LSOA_name,Type_I_crime_amount,Total_month,Type_II_crime_amount
0,2011,1,E01000001,City of London 001A,2,1,8
1,2011,1,E01000002,City of London 001B,2,1,17
2,2011,1,E01000003,City of London 001C,3,1,8
3,2011,1,E01000005,City of London 001E,7,1,21
4,2011,1,E01000254,Barnet 001D,1,1,1
...,...,...,...,...,...,...,...
3286689,2021,10,W01001954,Cardiff 006F,2,130,2
3286690,2021,10,W01001955,Swansea 025F,91,130,166
3286691,2021,10,W01001956,Swansea 023E,5,130,2
3286692,2021,10,W01001957,Swansea 025G,13,130,24


In [15]:
# Make sure all data is stored in the order of LSOA code, and then total month
# Then this will be the final dataset we will use
df_all = df_amount.sort_values(by = ['LSOA_code', 'Total_month'])
df_all.to_csv('all_variables.csv', index = False)

In [16]:
# These are the columns in the final dataset
df_all.columns

Index(['Year', 'Month', 'LSOA_code', 'LSOA_name', 'Type_I_crime_amount',
       'Total_month', 'Type_II_crime_amount'],
      dtype='object')

In [17]:
# Get the list of LSOA codes that has complete crime records over at least 10 years
df_all_data = df_all.groupby(['LSOA_code', 'LSOA_name']).count()[['Total_month']].reset_index()
df_all_data = df_all_data[df_all_data['Total_month'] >= 120]
df_all_data

,LSOA_code,LSOA_name,Total_month
1,E01000002,City of London 001B,126
3,E01000005,City of London 001E,130
1793,E01006412,Knowsley 006A,123
1794,E01006413,Knowsley 006B,125
1795,E01006414,Knowsley 006C,120
...,...,...,...
30105,W01001952,Cardiff 049F,130
30106,W01001953,Cardiff 005G,125
30108,W01001955,Swansea 025F,130
30110,W01001957,Swansea 025G,130


In [18]:
# Import external csv from the UK government about classification of LSOA code for urban, suburban and rural
# Merge it with usable LSOA code found above
df_urban = pd.read_csv("RUC_LSOA_2001_EW_LU.csv")
df_urban = df_urban[['LSOA01CD', 'Morphology Name']]
df_urban.columns = ['LSOA_code', 'Urban_type']
df_usable = pd.merge(df_all_data, df_urban, how = 'left', on = 'LSOA_code')
df_usable

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: DtypeWarning: Columns (0,1,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,LSOA_code,LSOA_name,Total_month,Urban_type
0,E01000002,City of London 001B,126,Urban > 10K
1,E01000005,City of London 001E,130,Urban > 10K
2,E01006412,Knowsley 006A,123,Urban > 10K
3,E01006413,Knowsley 006B,125,Urban > 10K
4,E01006414,Knowsley 006C,120,Urban > 10K
...,...,...,...,...
13470,W01001952,Cardiff 049F,130,NaN
13471,W01001953,Cardiff 005G,125,NaN
13472,W01001955,Swansea 025F,130,NaN
13473,W01001957,Swansea 025G,130,NaN


In [19]:
# Remove LSOA codes that are not shown in the classification list
df_final_usable = df_usable[~df_usable['Urban_type'].isna()]
df_final_usable.to_csv('LSOA_Urban_Type.csv', index = False)

In [20]:
# Determine 3 types of urban classifications by the government
df_usable['Urban_type'].unique().tolist()

['Urban > 10K', 'Village Hamlet & Isolated Dwellings', 'Town and Fringe', nan]